# Eagles CLV Refresh Analysis V1.0.1

# Install Required Packages

In [1]:
import pandas as pd
import numpy as np
import time

# import pickle as pickle

import os
from datetime import datetime


In [8]:
import boto3
# import datetime
import json
import logging
from io import StringIO
import sys
import time



from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

%matplotlib inline

pd.set_option('display.max_rows', 500)

# Functions

In [ ]:
def fix_col_names(df):
    '''
    takes a df and converts column names to lowercase underscore seperated
    '''
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    return df

# Input Data

# This data is saved out on the shared drive
# TODO: Add your input csv here
dat = pd.read_csv(r'C:\Users\rhead\Documents\local_ds\Model_refreshes\eagles\EAGLES_STAGE_CLVMODELOUTPUTFULL_7242024.csv')

In [ ]:

# Create df object that we will use for the rest of the script - preserve original dfclv
dfclv = fix_col_names(dat.copy())

# create date col for date logic
dfclv['date'] = pd.to_datetime(dfclv['modelrundate'], utc=True).dt.date
dfclv.sort_values(['audienceid','date'], inplace=True)


# Retention Graph

In [ ]:
import numpy as np
import pandas as pd

def create_retention_merged_df(dfclv):
    """
    Creates a merged DataFrame for Retention charts dividing the df into predictions and actuals and merging them
    after making some adjustments.

    This function processes the input DataFrame to identify customers 
    belonging to the 'STM' segment and checks if they have renewed their 
    status in the latest date. It adjusts deciles for the initial date 
    and merges initial and comparison DataFrames to produce the final 
    merged DataFrame with renewal information.

    Returns:
    Since to construct the retention chart we only require a few features from the main df
    the returning df will also retain a subset of columns
    pd.DataFrame: A DataFrame containing 'audienceid', 'probstm_x', 'renewed', 
                  'deciles_x', and 'date_x' with renewal status.
    """
    #Selet the required columns
    df = dfclv[['date', 'audienceid', 'clvcustomersegment', 'probstm', 'deciles']].copy()
    
    # select all model rundates except the actuals one
    df_initial = df[df.date != df.date.max()]
    
    # make sure we only have STMs since we are analyzing the STM retention model
    df_initial_stm = df_initial[df_initial.clvcustomersegment == 'STM']
    
    # this is an ad hoc fix because this model run date for Eagles had deciles from (0-9) instead of (1-10) like the rest of them
    df_initial_stm['deciles'] = np.where(df_initial_stm.date == df_initial_stm.date.min(), df_initial_stm.deciles + 1, df_initial_stm['deciles'])

    #create the actuals df
    df_comp = df[df.date == df.date.max()]

    #filter only the audienceids that are present in the predictions df
    df_comp = df_comp[df_comp.audienceid.isin(df_initial_stm.audienceid.values.tolist())]

    # create a renewed column for those customers who renewed their STM package
    df_comp['renewed'] = np.where(df_comp.clvcustomersegment == 'STM', 1, 0)

    #Merge both dataframes on audienceid
    df_merge = df_initial_stm.merge(df_comp, how='left', on='audienceid')[['audienceid', 'probstm_x', 'renewed', 'deciles_x', 'date_x']]

    #renewed will have 0 for the ones who churned and are not a part of the actuals dataframe
    df_merge['renewed'] = np.where(df_merge.renewed.isna(), 0, df_merge.renewed)
    return df_merge

In [ ]:
df_merge = create_retention_merged_df(dfclv)
df_merge.head()

In [ ]:

def create_final_retention_decile_graph(df_merge, get_data=False):
    """
    Displays retention graph for final slide deck along with additional labels for the mean
    probs and actuals for each model run and all model run plus actual results together
    
    input:
        df_merge: input dataframe
        get_data: True/False to have the data output to an object
    output:
        score_df:  optinal output of data
    """
    df_all = pd.DataFrame() #for the overall comp chart
    
    dates = df_merge.date_x.value_counts().index.sort_values()
    
    for i in dates:
        print(i)
        fig, ax = plt.subplots(figsize = (12, 6))
        
        # Create subsets grouping by the different deciles
        deciles = df_merge[df_merge.date_x == i].groupby('deciles_x')['probstm_x'].mean()
        score_df = deciles.to_frame()
        score_df['deciles_x'] = score_df.index
        score_df.reset_index(drop=True, inplace=True)
    
        rfactual = df_merge[df_merge.date_x == i].groupby('deciles_x')['renewed'].mean()
        rfactual.reset_index(drop=True, inplace=True)
        score_df = pd.concat((score_df, rfactual), axis = 1)
        score_df.rename(columns={'probstm_x':"Model Prediction", 'renewed':'Actual Results'}, inplace=True)
       
        #Reformat df shape to use in plot
        df_forplot = pd.melt(score_df, id_vars=['deciles_x'], value_vars=['Model Prediction', 'Actual Results'], var_name='Model', value_name = 'Probability')
        
        ### Create Plot
        ax= sns.lineplot(x="deciles_x", y="Probability", hue="Model",
                          palette= ['#1f497d', '#4290dc'], marker="D", data=df_forplot) #'#b3d3f1'
        plt.axhline(y=score_df['Actual Results'].mean(), color='#4290dc',ls='--')
        plt.axhline(y=score_df['Model Prediction'].mean(),color='#1f497d',ls='--')
        ax.set(ylim=(0, 1.1))
        plt.legend(loc='lower right')
        plt.show()
    
        print('___________________________________')
        print('mean values for header and graph')
        display(np.round(score_df[['Model Prediction','Actual Results']].mean()*100, 2))
        print('_______________________________________________________________________________________________________________________')
    
        if get_data:
            score_df['date'] = i
            df_all = pd.concat([df_all, score_df])
            print(score_df)
    # Get a retention decile plot for all probabilities together
    actuals_avg = df_all.groupby('deciles_x')['Actual Results'].mean()

    #Reformat df shape to use in plot
    df_all = df_all.pivot(index = 'date', columns = 'deciles_x', values = 'Model Prediction')
    print('_______________________________________________________________________________________________________________________')
    actual_avg_df = pd.DataFrame(actuals_avg.values).T
    actual_avg_df.columns = range(1,11)
    actual_avg_df.index = ['Actual Results']
    df_allplot = pd.concat([df_all, actual_avg_df])
    df_allplot = df_allplot.reset_index()

    df_allplot_long = pd.melt(df_allplot, id_vars='index', value_vars=[1,2,3,4,5,6,7,8,9,10], var_name='Decile',
                              value_name = 'Probability')
    fig, ax = plt.subplots(figsize = (12, 6))
    ax= sns.lineplot(x="Decile", y="Probability", hue="index", marker="D", palette = "Blues", data=df_allplot_long) #'#b3d3f1'

    # Get the lines from the plot
    lines = ax.get_lines()
    
    # Change the color of the specific line (e.g., the second line) to orange
    for line, label in zip(lines, ax.get_legend().get_texts()):
        if label.get_text() == 'Actual Results':
            line.set_color('orange')

    # Update the legend to reflect the color change
    new_labels = [text.get_text() for text in ax.get_legend().get_texts()]
    new_colors = [line.get_color() for line in lines]
    
    # Remove the old legend
    ax.get_legend().remove()
    
    # Create a new legend with updated colors
    ax.legend(handles=ax.get_lines(), labels=new_labels)
    
    ax.set(ylim=(0.8, 1.1))
    # plt.legend(loc='lower right')
    plt.show()

In [ ]:
create_final_retention_decile_graph(df_merge, True)

### Brier Score

In [ ]:
#TODO: brier score from sklearn can also be used
df_merge['diff_sqrd'] = np.subtract(df_merge.renewed, df_merge.probstm_x)**2

In [ ]:
#Brier scores per model rundate
df_merge.groupby(['date_x'])['diff_sqrd'].mean() #brier score per model run date

#### MSE mean probs vs actuals per decile
This method was done to test which process makes more sense. The below method averages the probabilities across deciles and averages actual renewal across deciles and then the mean squared error between them are calculated. This method scores were not a part of the final analysis put forward to the client because the averaging probabilities out on a decile loses valuable informatin for that decile. Instead calculating diff_squared per customer and averaging it out per a model run date (as above) reserves information of each customer and provides accurate information to compare model runs

In [ ]:
dates = df_merge.date_x.value_counts().index.sort_values()
for i in dates:
    print(i)
    deciles = df_merge[df_merge.date_x == i].groupby('deciles_x')['probstm_x'].mean()
    score_df = deciles.to_frame()
    score_df['deciles'] = score_df.index
    score_df.reset_index(drop=True, inplace=True)
    
    rfactual = df_merge.groupby('deciles_x')['renewed'].mean()
    rfactual.reset_index(drop=True, inplace=True)
    score_df = pd.concat((score_df, rfactual), axis = 1)
    score_df.rename(columns={'probstm_x':"Model Prediction", 'renewed':'Actual Results'}, inplace=True)
    score_df['brier_score_deciles'] = np.subtract(score_df['Actual Results'], score_df['Model Prediction'])**2

    print(score_df[['deciles', 'brier_score_deciles']])

### Confusion Matrices

def create_df_confusion_matrix(dfclv):
    """
    Creates a DataFrame which will be an input to the confusion matrices.
    The dataframew will be a subset of the main dataframe with required columns.

    Creates a predicted class column which is the max(column name) out of all the probability columns and then maps
    it out the the correct segment

    Parameters:
    dfclv (pd.DataFrame): Input DataFrame containing customer data with columns 
                          'date', 'audienceid', 'clvcustomersegment', 'probstm', 
                          'probsgb', 'probsecondary', 'probchurn', and 'deciles'.

    Returns:
    pd.DataFrame: A DataFrame containing 'audienceid', 'predicted_class', 'actual_class', 
                  and 'date' with predicted and actual customer segments.
    """
    #filter out the required columns
    df = dfclv[['date', 'audienceid', 'clvcustomersegment', 'probstm', 'probsgb', 'probsecondary', 'probchurn', 'deciles']].copy()

    #separate initial model run dates and the actual model run date
    df_initial = df[df.date != df.date.max()]

    df_initial_stm = df_initial[df_initial.clvcustomersegment == 'STM']

    #map the max probability class to the segment
    map_classes = {'probstm': 'STM', 'probsgb': 'Churn', 'probsecondary': 'Churn', 'probchurn': 'Churn'}
    df_initial_stm["predicted_class"] = df_initial_stm[['probstm', 'probsgb', 'probsecondary', 'probchurn']].idxmax(axis=1)
    df_initial_stm["predicted_class"] = df_initial_stm["predicted_class"].map(map_classes)

    #actuals df
    df_comp = df[df.date == df.date.max()]
    df_comp = df_comp[df_comp.audienceid.isin(df_initial_stm.audienceid.values.tolist())]

    #merge dfs
    
    df_merge = df_initial_stm.merge(df_comp, how='left', on='audienceid')[['audienceid', 'predicted_class', 'clvcustomersegment_y', 'date_x']]
    df_merge = df_merge.rename(columns={'clvcustomersegment_y': 'actual_class', 'date_x': 'date'})
    
    #churn where the actual class is na
    actuals_retention_map = {'STM': 'STM', 'SGB': 'Churn', 'Secondary': 'Churn', 'Churn': 'Churn'}
    df_merge['actual_class'] = np.where(df_merge.actual_class.isna(), 'Churn', df_merge.actual_class)
    df_merge["actual_class"] = df_merge["actual_class"].map(actuals_retention_map)
    
    return df_merge

In [ ]:
df_confusion_matrix = create_df_confusion_matrix(dfclv)

In [ ]:
# Function to generate confusion matrix for a given segment
segments = ['STM', 'Churn']
def generate_confusion_matrix(date):
    actual = df_confusion_matrix[df_confusion_matrix.date == date]['actual_class']
    predicted = df_confusion_matrix[df_confusion_matrix.date == date]['predicted_class']
    cm = confusion_matrix(actual, predicted, labels=segments)
    return cm

dates = df_confusion_matrix.date.value_counts().index.sort_values()

# Generate and plot confusion matrices for each segment
for i in dates:
    cm = generate_confusion_matrix(i)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=segments, yticklabels=segments)
    plt.title(f'Confusion Matrix for {i}')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

# Premium STM Analysis
The below analysis is Eagles specific and has nothing to do with Model Performance Analysis

## CLV Histogram

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
sns.set(font='Arial', style = 'white')

In [ ]:
# TODO: Can add seasonyear filter
def create_dist_graph(MAX_AXS, SERIES_1, SERIES_2=False, save_file=False, SAVEFILEPATH=''):
    """
    This function creates the distribution graph shared out with clients of the segments finalclv with vertical 
    markers for central tendency.  There's an option to include a second subseries to note addition central tendency,
    usually used for premium within the stm segment. Best practice would be to have save_file parameter set to false 
    while you adjust the max value of the X-axis until the graph looks good.  Then run again with save_file=True which
    will save a png out to the drive. 
    inputs:
        MAX_AXS - Due to outliers, set this value such that the graph shows an informative distribution
        SERIES_1 - a pandas series of just finalclv, after segmentation
        SERIES_2 - a 2nd pandas series that a subset of the first, used to mark additional cental tendency markers
        save_file - default is false while trial and error is performed to get a reasonable looking distribution to 
            report out
        SAVEFILEPATH - the name and path for the final png file of the distribution
    outputs:
        - display of graph, various stats, and a png file written out to a specified location
    
    """
        
    #get obs cnt to show how many are off the chart after adjusting max_axs parm
    obs_cnt = SERIES_1.count()
    off_chart_obs_cnt = SERIES_1[SERIES_1 > MAX_AXS].count()
        
    print('slide detail to add to final slide')
    print('Series_1')
    print('_'*20)
    print('Total:\t',"{:,}".format(np.round(SERIES_1.sum(),2)))
    print('Median:\t',"{:,}".format(np.round(SERIES_1.median(),2)))
    print('Mean:\t',"{:,}".format(np.round(SERIES_1.mean(),2)))
    print('Max:\t',"{:,}".format(np.round(SERIES_1.max(),2)))
    print('Count Obs > Max_Axs:\t',"{:,} {:,}%".format(off_chart_obs_cnt, np.round(off_chart_obs_cnt/obs_cnt*100,2)))
    print('_'*40)
    
    fig, ax = plt.subplots(figsize = (12, 6))
    sns.distplot(SERIES_1, kde = False, bins = np.linspace(0, MAX_AXS, 50), \
                 color = '#1F497D', hist_kws=dict(alpha = 1, linewidth = 0))
    sns.distplot(SERIES_2, kde = False, bins = np.linspace(0, MAX_AXS, 50), \
                 color = 'turquoise', hist_kws=dict(alpha = 1, linewidth = 0))
    sns.despine()
    ax.set_xlim(0, MAX_AXS)
    
    # vertical flags for central tendency
    ax.axvline(x=SERIES_1.mean(), color = '#7ea7f0')
    ax.axvline(x=SERIES_1.median(), color = '#7ea7f0')
    
    # vertical flags for central tendency if 2nd series exists
    if type(SERIES_2) == pd.core.series.Series:
        
        print('Series_2')
        print('_'*20)
        print('Total:\t',"{:,}".format(np.round(SERIES_2.sum(),2)))
        print('Median:\t',"{:,}".format(np.round(SERIES_2.median(),2)))
        print('Mean:\t',"{:,}".format(np.round(SERIES_2.mean(),2)))
        print('_'*40)
    
        ax.axvline(x=SERIES_2.mean(), color = 'grey')
        ax.axvline(x=SERIES_2.median(), color = 'grey')
        
    #finish formatting graph
    ax.set_xlabel("3 Year Predicted CLV", fontsize = 14)
    ax.set_ylabel("Number of Customers", fontsize = 14)
    xlabels = ['$' + '{:,.0f}'.format(x) for x in ax.get_xticks()]
    ax.set_xticklabels(xlabels)
    plt.tick_params(labelsize = 14)
    
    if save_file:
        print('saving file to:  ',SAVEFILEPATH)
        plt.savefig(SAVEFILEPATH)

In [ ]:
SEGMENT = 'STM'

seg_finalclv = dfclv[(dfclv.clvcustomersegment==SEGMENT) & (dfclv.currentpremium==0) & (dfclv.modelrundate == '2024-02-12 00:00:00.000000000 -05:00')]['finalclv']
subseg_finalclv = dfclv[(dfclv.clvcustomersegment==SEGMENT) & (dfclv.currentpremium==1) & (dfclv.modelrundate == '2024-02-12 00:00:00.000000000 -05:00')]['finalclv']

create_dist_graph(MAX_AXS=90000, SERIES_1=seg_finalclv, SERIES_2=subseg_finalclv, save_file=False, SAVEFILEPATH= '')

In [ ]:
dfclv[dfclv.finalclv == seg_finalclv.max()]

## Violin Plots

# Creating spend buckets
df = dfclv[(dfclv.clvcustomersegment == 'STM') & (dfclv.modelrundate == '2024-02-12 00:00:00.000000000 -05:00')].copy()

# Creating spend buckets
bins = [800, 2000, 3000, 4000, 5000, 6000, 8000]
labels = [f'{bins[i]}-{bins[i+1]}' for i in range(len(bins)-1)]
df['spend_bucket'] = pd.cut(df['totaleaglesspend'], bins=bins, labels=labels, include_lowest=True)

# Creating the violin plot
g = sns.catplot(data=df, x = 'spend_bucket', hue = 'currentpremium'
                , y='finalclv', kind='violin', height=6.45, aspect=1.55)

In [ ]:
# Creating spend buckets
df = dfclv[(dfclv.clvcustomersegment == 'STM') & (dfclv.modelrundate == '2024-02-12 00:00:00.000000000 -05:00') & (dfclv.finalclv < 200000)].copy()

# Creating spend buckets
bins = [0, 0.2, 0.3, 0.4, 0.5, 0.7, 0.8, 1]
labels = [f'{bins[i]}-{bins[i+1]}' for i in range(len(bins)-1)]
df['utilization_bucket'] = pd.cut(df['utilizationrate'], bins=bins, labels=labels, include_lowest=True)

# Creating the violin plot
g = sns.catplot(data=df, x = 'utilization_bucket', hue = 'currentpremium'
                , y='finalclv', kind='violin', height=6.45, aspect=1.55)

# Setting y-axis limits to ensure no negative values are shown
# g.set(ylim=(-1, df['finalclv'].max()))

In [ ]:
# Creating spend buckets
df = dfclv[(dfclv.clvcustomersegment == 'STM') & (dfclv.modelrundate == '2024-02-12 00:00:00.000000000 -05:00') & (dfclv.finalclv < 200000)].copy()

# Creating spend buckets
bins = [0, 10, 50, 100, 200, 6940]
labels = [f'{bins[i]}-{bins[i+1]}' for i in range(len(bins)-1)]
df['ticketstransferred_bucket'] = pd.cut(df['ticketstransferred'], bins=bins, labels=labels, include_lowest=True)

# Creating the violin plot
g = sns.catplot(data=df, x = 'ticketstransferred_bucket', hue = 'currentpremium'
                , y='finalclv', kind='violin', height=6.45, aspect=1.55)

# Setting y-axis limits to ensure no negative values are shown
# g.set(ylim=(-1, df['finalclv'].max()))